# Robot Painter

(circa 11/23/22) Code taken from Elaine Pham's work for and the starter code from https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/18ba0481-7e30-46ba-9e7d-f7b34500d6dc/%2Frobot_painter.ipynb

intermediate function source (circa 11/25/22 https://stackoverflow.com/questions/43594646/how-to-calculate-the-coordinates-of-the-line-between-two-points-in-python)

Changes to make straight lines going to a center point instead of a circle and to save those.

In [2]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from manipulation import running_as_notebook, FindResource
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import MakeManipulationStation
from pydrake.all import (AddMultibodyPlantSceneGraph, AngleAxis, BasicVector,
                         ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant,
                         MultibodyPositionToGeometryPose, Parser,
                         PiecewisePose, Quaternion, RigidTransform,
                         RollPitchYaw, RotationMatrix, SceneGraph, Simulator,
                         StartMeshcat, TrajectorySource)
import numpy as np
import torch 

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://a569df8b-a69d-4307-8cbd-4f24b608e3b4.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


In [3]:
class IIWA_Painter():
    def __init__(self, traj=None, init_angles=None):
        builder = DiagramBuilder()
        # set up the system of manipulation station
        self.station = MakeManipulationStation(filename=FindResource("models/iiwa_and_wsg.dmd.yaml"))

        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")

        # optionally add trajectory source
        if traj is not None:
            traj_V_G = traj.MakeDerivative()
            V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
            self.controller = builder.AddSystem(
                PseudoInverseController(self.plant))
            builder.Connect(V_G_source.get_output_port(),
                            self.controller.GetInputPort("V_G"))

            self.integrator = builder.AddSystem(Integrator(7))
            builder.Connect(self.controller.get_output_port(),
                            self.integrator.get_input_port())
            builder.Connect(self.integrator.get_output_port(),
                            self.station.GetInputPort("iiwa_position"))
            builder.Connect(
                self.station.GetOutputPort("iiwa_position_measured"),
                self.controller.GetInputPort("iiwa_position"))

        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat, params)

        wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(wsg_position.get_output_port(),
                        self.station.GetInputPort("wsg_position"))

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext(init_angles)
        self.diagram.Publish(context)

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies
        
        Input: 
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.
        
        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(meshcat, "painter/" + name,
                        length=length, radius=radius, X_PT=X_WF)

    def CreateDefaultContext(self, init_angles=None):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context)

        if init_angles is not None:
            q0 = init_angles
        else:
            # provide initial states
            q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                        -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])        

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)

        return context


    def get_X_WG(self, context=None):

        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG

    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(1.0)

        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)

class PseudoInverseController(LeafSystem):
    """
    same controller seen in-class
    """
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_G", BasicVector(6))
        self.q_port = self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7),
                                     self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G) #important
        output.SetFromVector(v)

def compose_circular_key_frames(thetas, X_WorldCenter, X_WorldGripper_init, radius):
    """    
    returns: a list of RigidTransforms
    """

    ## this is an template, replace your code below
    # key_frame_poses_in_world = [X_WorldCenter @ X_WorldGripper_init]
    for i in range(len(thetas)):
        rotate = RigidTransform(RotationMatrix.MakeYRotation(thetas[i]));
        translate = RigidTransform([0, 0, -radius])
        this_pose = RigidTransform(X_WorldCenter @ rotate @ translate)
        key_frame_poses_in_world.append(this_pose)
        
    return key_frame_poses_in_world

# check key frames instead of interpolated trajectory
def visualize_key_frames(painter,frame_poses):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame('frame_{}'.format(i), pose, length=0.05)


In [4]:
### helper function to run and visualize the poses outputted by the NN ###

def run_simulation(times, key_frame_poses):
    """ create a traj from the key_frame_poses and run simulation """
    results = []

    # creates a trajectory out of the poses 
    traj = PiecewisePose.MakeLinear(times, key_frame_poses)
    # visualize the traj
    painter = IIWA_Painter(traj)
    visualize_key_frames(painter, key_frame_poses)

    # run sim
    context = painter.CreateDefaultContext()
    plant_context = painter.plant.GetMyContextFromRoot(context)
    iiwa = painter.plant.GetModelInstanceByName("iiwa")
    simulator = Simulator(painter.diagram, context)
    simulator.set_target_realtime_rate(1.0)
    for timestep in times:
        simulator.AdvanceTo(timestep)
        gripper = painter.plant.GetBodyByName("iiwa_link_7")
        pose = np.ndarray.tolist(gripper.EvalPoseInWorld(plant_context).translation())
        angles = np.ndarray.tolist(painter.plant.GetPositions(plant_context, iiwa))
        current_pose_data = {
            'timestep' : timestep,
            'joint_angle' : angles,
            'end_effector_position' : pose,
        }
        results.append(current_pose_data)
    
    return results

In [7]:
### Run this cell to feed inputs into the NN and display the outputs in simulation ###

# create iiwa and get init gripper pose
painter = IIWA_Painter()
X_WG_init = painter.get_X_WG()

# NN
model = torch.nn.Linear(3,3) # TODO: replace this with model saved in disk
model = model.float()
steps = 1 # num of inputs we will give the NN

# simulation params
num_key_frames = steps
total_time = 20
times = np.linspace(0, total_time, num_key_frames+1)

guess = test_inputs[0] # TODO: change this to be actual test input
key_frame_poses = [guess]
# gets the output from the NN 
for step in range(steps):
    xyz = torch.from_numpy(guess.translation()).float()
    pred = model(xyz)
    key_frame_poses.append(RigidTransform(
                                guess.rotation(), 
                                pred.detach().numpy()))
    guess = pred

results = run_simulation(times, key_frame_poses)        


NameError: name 'pose_index' is not defined

In [5]:
### Sample NN inputs; circular key frame poses from painter robot homework ###

test_inputs = [RigidTransform(
  R=RotationMatrix([
    [-2.3403549959129465e-06, 0.05032888698822864, -0.9987326985380272],
    [0.9999999999051579, -1.3437349077360507e-05, -3.0204696645260106e-06],
    [-1.3572336781715093e-05, -0.9987326984503733, -0.05032888695200715],
  ]),
  p=[0.4686663152509269, -1.0643346551004039e-05, 0.6012858601360346],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 0.0, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.6427876096865393, 0.0, -0.766044443118978],
    [0.766044443118978, 0.0, 0.6427876096865393],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.5266044443118978, -0.06427876096865393, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.984807753012208, 0.0, -0.17364817766693041],
    [0.17364817766693041, 0.0, 0.984807753012208],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.46736481776669303, -0.0984807753012208, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.8660254037844387, 0.0, 0.4999999999999998],
    [-0.4999999999999998, 0.0, 0.8660254037844387],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.4, -0.08660254037844388, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.3420201433256689, 0.0, 0.9396926207859083],
    [-0.9396926207859083, 0.0, 0.3420201433256689],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, -0.03420201433256689, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.34202014332566866, 0.0, 0.9396926207859084],
    [-0.9396926207859084, 0.0, -0.34202014332566866],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, 0.034202014332566866, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.8660254037844384, 0.0, 0.5000000000000004],
    [-0.5000000000000004, 0.0, -0.8660254037844384],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.39999999999999997, 0.08660254037844384, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.9848077530122081, 0.0, -0.17364817766692997],
    [0.17364817766692997, 0.0, -0.9848077530122081],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.46736481776669303, 0.09848077530122082, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.6427876096865396, 0.0, -0.7660444431189778],
    [0.7660444431189778, 0.0, -0.6427876096865396],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.5266044443118978, 0.06427876096865397, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-2.4492935982947064e-16, 0.0, -1.0],
    [1.0, 0.0, -2.4492935982947064e-16],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 2.4492935982947065e-17, 0.4],
),
  RigidTransform(
  R=RotationMatrix([
    [-2.4492935982947064e-16, 0.0, -1.0],
    [1.0, 0.0, -2.4492935982947064e-16],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 2.4492935982947065e-17, 0.4],
)]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a569df8b-a69d-4307-8cbd-4f24b608e3b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>